## Выделение QRS комплекса в записях

In [ ]:
%%capture
!gdown 1gU56tHhpG5K7-AKBv8inJxRYLZAz1Gib
!gdown 1Tr9yHkmfDvc_VuZQ6IJXeZ08x7YbyPP_

!unzip ptbxl_full_signals.zip

In [ ]:
!pip install neurokit2 biosppy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from tqdm import tqdm
import os
from scipy.signal import resample
from biosppy.signals.ecg import christov_segmenter
from IPython.display import clear_output

from scipy import spatial
from scipy.signal import savgol_filter

In [ ]:
labels = pd.read_csv('/content/ptblx_full_signals.csv')
labels['record_name'] = labels.filename_hr.apply(lambda x: x[-8:])

labels = labels[['ecg_id', 'record_name', 'result_class']]
new_labels = pd.DataFrame(columns=['record_name', 'result_class'])

In [ ]:
!mkdir r_peaks
!mkdir r_peaks/signals

In [ ]:
latest = []

for ecg in tqdm(labels.ecg_id[24461:]):
    name = labels.loc[labels.ecg_id==ecg].record_name.values[0]
    data = np.load(f'ptbxl_full_signals/signals/{name}.npy')

    data = np.apply_along_axis(lambda x: nk.ecg_clean(x, sampling_rate=500), axis=1, arr=data)

    peaks = [nk.ecg_findpeaks(data[i])['ECG_R_Peaks'] for i in range(12)]
    peaks_count = [(i, len(i)) for i in peaks]

    peaks = max(peaks_count, key=lambda x: x[1])[0]
    if len(peaks) < 2:
        continue

    signals = []
    if name not in latest:
        for count, i in enumerate(peaks):
            if count == 0:
                diff2 = abs(peaks[count + 1] - i)
                x = 0
                y = peaks[count + 1] - diff2 // 2
            elif count == len(peaks)-1:
                diff1 = abs(peaks[count - 1] - i)
                x = peaks[count - 1] + diff1 // 2
                y = 5000
            else:
                diff1 = abs(peaks[count - 1] - i)
                diff2 = abs(peaks[count + 1]- i)
                x = peaks[count - 1] + diff1 // 2
                y = peaks[count + 1] - diff2 // 2

            signal = resample(data[:, x:y], 500, axis=1)

            signals.append(signal)

        for i, signal in enumerate(signals):
            new_labels.loc[len(new_labels.index)] = [f'{name}_{i}', labels[labels.ecg_id == ecg].result_class.values[0]]
            np.save(f'r_peaks/signals/{name}_{i}.npy', signal)
    else:
        df = new_labels[new_labels.record_name.apply(lambda x: x[:8] == name)].copy()
        df.result_class = [labels[labels.ecg_id == ecg].result_class.values[0]]*len(df)

        new_labels = pd.concat([new_labels, df])

    latest.append(name)

100%|██████████| 3396/3396 [20:36<00:00,  2.75it/s]


In [ ]:
new_labels.to_csv('r_peaks/labels.csv', index=False)

In [ ]:
!zip -r r_peaks.zip r_peaks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mv r_peaks.zip /content/drive/MyDrive/cardioScreen/data/

Mounted at /content/drive
